Import packages

In [1]:
import cv2
import numpy as np
import math
import mediapipe as mp
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume

In [2]:
mpHands = mp.solutions.hands
hands = mpHands.Hands(static_image_mode=False,
                      max_num_hands=2,
                      min_detection_confidence=0.7,
                      min_tracking_confidence=0.5)
mpDraw = mp.solutions.drawing_utils

devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(
    IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))
volRange=volume.GetVolumeRange()
vol=volume.GetMasterVolumeLevel()

cap=cv2.VideoCapture(0)
cap.set(3,700)
cap.set(4,700)
pTime=0
while True:
    lmList=[]
    success,img=cap.read()
    img=cv2.flip(img,1)
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB)

    if results.multi_hand_landmarks:
        for handLms in results.multi_hand_landmarks:
            for id, lm in enumerate(handLms.landmark):
                #print(id,lm)
                h, w, c = img.shape
                
                cx, cy = int(lm.x *w), int(lm.y*h)
                lmList.append([id,cx,cy])
            #mpDraw.draw_landmarks(img, handLms, mpHands.HAND_CONNECTIONS)
    if(len(lmList)>0):
        cv2.circle(img, (lmList[4][1],lmList[4][2]), 5, (255,0,255), cv2.FILLED)
        cv2.circle(img, (lmList[8][1],lmList[8][2]), 5, (255,0,255), cv2.FILLED)
        cv2.line(img,(lmList[4][1],lmList[4][2]),(lmList[8][1],lmList[8][2]),(255,0,255),3)
        length=math.hypot(lmList[4][1]-lmList[8][1],lmList[4][2]-lmList[8][2])
        #print(length,volRange[0])
        vol=np.interp(length,[14,140],[volRange[0],volRange[1]])
        volume.SetMasterVolumeLevel(vol, None)
        #volume range -96 0
        #length 140  14

    
    volBar=np.interp(vol,[-96,0],[100,0])
    cv2.rectangle(img, (0,int(volBar)),(30,200), (255,0,255), cv2.FILLED)
    cv2.imshow("Volume Control",img)
    cv2.waitKey(1)
